In [2]:
import pandas as pd
from tabulate import tabulate
import requests, json, sys, os, re

# Ensure the utils package is installed and accessible
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

#import local packages
from plumbing.natstat_ingestion import ingest_teams_data, ingest_games_data, parse_game_data

In [ ]:
# TEST URL OUTPUT
# url = f"https://interst.at/game/pfb/2024"
# response = requests.get(url)
# response.raise_for_status()
# data = response.json()
# pretty_data = json.dumps(data, indent=3)
# #print(pretty_data)

In [7]:
game_df = ingest_games_data()
team_df = ingest_teams_data()

In [ ]:
game_df = game_df[game_df['status'] == 'Final']
print(tabulate(team_df, headers='keys', tablefmt='grid'))

In [ ]:
#TODO: Write a games loop that loops through all (active only?) games in game_df and calls the (game and player) parsing functions

from plumbing.natstat_ingestion import ingest_teams_data, ingest_games_data, parse_game_data

# INGEST
team_df = ingest_teams_data()
game_df = ingest_games_data()

team_df = team_df.style.background_gradient(cmap='viridis')

# Get list of finished games and their URLs
final_games_df = game_df[game_df['status'] == 'Final']
finished_games_urls_list = final_games_df['game_api_url'].tolist()
# Initialize game_stat and player_stat dataframes to append each game to 
game_stat_df = pd.DataFrame()
player_stat_df = pd.DataFrame()
error_log = []

for game_url in finished_games_urls_list:
    print(f"Game URL: {game_url}")
    try:
        # Append the parsed game and player data to the respective dataframes
        game_data, player_data = parse_game_data(game_url)
        print(player_data)
        game_stat_df = pd.concat([game_stat_df, game_data], ignore_index=True)
        
        # player_data = parse_player_data(data)
        # player_stat_df = pd.concat([player_stat_df, player_data], ignore_index=True)
    except requests.RequestException as e:
        print(f"Error with URL {game_url}: {e}")
        error_log.append((game_url, str(e)))
    
# Print or save the error log for further inspection
print("Error Log:", error_log)


In [ ]:
print(tabulate(game_stat_df, headers='keys', tablefmt='grid'))

In [ ]:
# def enrich_games_data(games_df):
#     """Takes a dataframe of games and parses the data for each game.

#     Args:
#         game_url (_type_): _description_
#     """
#     # Get list of finished games and their URLs
#     final_games_df = game_df[game_df['status'] == 'Final']
#     finished_games_urls_list = final_games_df['game_api_url'].tolist()

#     # Initialize game_stat and player_stat dataframes to append each game to 
#     game_stat_df = pd.DataFrame()
#     player_stat_df = pd.DataFrame()

#     for game_url in finished_games_urls_list:
#         game_url = f"https://interst.at{game_url}"
#         response = requests.get(game_url)
#         response.raise_for_status()
#         data = response.json()
#         # Append the parsed game and player data to the respective dataframes
#         pd.concat([game_stat_df, parse_game_data(data)], ignore_index=True)
#         pd.concat([player_stat_df, parse_player_data(data)], ignore_index=True)
    
#     return game_stat_df, player_stat_df

# gamestat_df, playerstat_df = enrich_games_data(data)